In [429]:
import pandas as pd
import os as _dir

working_dir = _dir.getcwd()
csv_file = "purchase_data.csv"

for root, dirs, files in _dir.walk(working_dir):
    if csv_file in files:
        csv_file = (_dir.path.join(root, csv_file))
purchase_main_df = pd.read_csv(csv_file)
purchase_df = purchase_main_df.copy()


## Player Count

* Displays the total number of players

In [430]:
total_player = pd.DataFrame([purchase_df["SN"].nunique()], columns=["Total Players"])
total_player

,Total Players
0,576


## Output of Purchasing Analysis (Total)

* Gives an analysis of the following pieces of data:
    * Number of unique items sold
    * The Average price
    * Total number of purchases
    * Total Revenue made from purchases


In [431]:
purchasing_data_list = [purchase_df["Item ID"].nunique(),"${:.2f}".format(purchase_df["Price"].mean()),purchase_df["Purchase ID"].count(),"${:,.2f}".format(purchase_df["Price"].sum())]
purchasing_totals_df = pd.DataFrame([purchasing_data_list], columns=['Number of unique items','Average Price','Number of Purchases','Total Revenue'])
purchasing_totals_df

,Number of unique items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Count and Percentage of Male Players


* Count and Percentage of Female Players


* Count and Percentage of Other / Non-Disclosed

In [432]:
Gender_Groupby = purchase_df.groupby("Gender")
Gender_count = Gender_Groupby["SN"].nunique()
Demographics_data = pd.DataFrame({"Total Counts": Gender_count,"Percentage of Players": (Gender_count/int(total_player.values))}, index=purchase_df["Gender"].unique()).sort_values(by=['Percentage of Players'], ascending=False)
Demographics_data["Percentage of Players"] = Demographics_data["Percentage of Players"].map("{:.2%}".format)
Demographics_data

,Total Counts,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Output of Purchasing Analysis (Gender)

* Gives an analysis of the following pieces of data aggregated by Gender:
    * Purchase Count
    * Average Purchase Price
    * Total Purchase Value
    * (Avg) Total Purchase /Person



In [433]:
alist = list(zip(Gender_Groupby["Purchase ID"].count(),Gender_Groupby["Price"].mean(),Gender_Groupby["Price"].sum(),round(Gender_Groupby["Price"].sum()/Gender_count,2)))
purchase_analysis_byGender = pd.DataFrame(alist,columns=['Purchase Count','Average Purchase Price','Total Purchase Value','(Avg) Total Purchase /Person'],index=Gender_Groupby['Gender'].unique().values).sort_values(by=['Purchase Count'], ascending=False)
for x in range(1,4):
    df_col = purchase_analysis_byGender.columns[x]
    purchase_analysis_byGender[df_col] = purchase_analysis_byGender[df_col].map("${:,.2f}".format)
purchase_analysis_byGender

,Purchase Count,Average Purchase Price,Total Purchase Value,(Avg) Total Purchase /Person
[Male],652,$3.02,"$1,967.64",$4.07
[Female],113,$3.20,$361.94,$4.47
[Other / Non-Disclosed],15,$3.35,$50.19,$4.56


## Bin Age Demographics

* Established bins for ages


* Categorized the existing players into the bin.


* Calculated the numbers and percentages by age group

In [434]:
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
purchase_df["Age Bin"] = pd.cut(purchase_df['Age'],bins, labels=labels)
age_demo_groupby = purchase_df.groupby("Age Bin")
age_count = age_demo_groupby["SN"].nunique()
agelist = list(zip(age_count,(age_count/int(total_player.values))))
age_demographics_df = pd.DataFrame(agelist,columns=['Total Count','Percentage of Players'],index=purchase_df["Age Bin"].unique().categories)
age_demographics_df["Percentage of Players"] = age_demographics_df["Percentage of Players"].map("{:.2%}".format)
age_demographics_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Bin Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Ran calculations for:
    * purchase count
    * avg. purchase price
    * avg. purchase total per person

In [435]:
age_purchase_list = list(zip(age_demo_groupby["Purchase ID"].count(),round(age_demo_groupby["Price"].mean(),2),round(age_demo_groupby["Price"].sum(),2),round((age_demo_groupby["Price"].sum()/age_count),2)))
age_purchase_df = pd.DataFrame(age_purchase_list,columns=['Purchase Count','Average Purchase Price','Total Purchase Value','Total Purchase /Person (Avg) '], index=purchase_df["Age Bin"].unique().categories)
for x in range(1,4):
    age_df_col = age_purchase_df.columns[x]
    age_purchase_df[age_df_col ] = age_purchase_df[age_df_col].map("${:,.2f}".format)
age_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Total Purchase /Person (Avg)
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Obtained the top (5) Spenders
* Sorted results by (Total Purchase Value)
* Cleaned up formatting by removing index header to table is even

In [436]:
sn_groupby = purchase_df.groupby("SN")
ilist = sn_groupby["Price"].sum().sort_values(ascending=False).head().index.values
top_spender_df = pd.DataFrame()
for i in ilist:
    top_spender_df = top_spender_df.append(purchase_df.loc[purchase_df["SN"] == i])
sn_groupby = top_spender_df.groupby("SN")
toplist = list(zip(sn_groupby["Purchase ID"].nunique(),round(sn_groupby["Price"].mean(),2),round(sn_groupby["Price"].sum(),2)))
top_spender_df = pd.DataFrame(toplist,columns=['Purchase Count','Purchase Price(Avg)','Total Purchase Value']).sort_values(by=['Total Purchase Value'],ascending=False)
top_spender_df.set_index(ilist)

,Purchase Count,Purchase Price(Avg),Total Purchase Value
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

* Grouped by (Item ID and Item Name)

* Created secondary df (index_df) as an index reference to use for formatting

* Created new df based on calculations

* Dynamically set currency formatting with for loop

* Combined (Item ID and Item Name) into one index column and removed header for formatting


In [437]:
item_grouping = purchase_df.groupby(["Item ID","Item Name"])
index_df = pd.DataFrame(item_grouping['Purchase ID'].nunique()).sort_values(by=['Purchase ID'],ascending=False).head()
product_list = list(zip(item_grouping["Purchase ID"].nunique(),item_grouping["Price"].mean(),item_grouping["Price"].sum()))
popular_items_df = pd.DataFrame(product_list,columns=['Purchase Count','Item Price','Purchase Value']).sort_values(by=['Purchase Count'],ascending=False).head()
for x in range(1,3):
    pop_df_col = popular_items_df.columns[x]
    popular_items_df[pop_df_col ] = popular_items_df[pop_df_col].map("${:,.2f}".format)
popular_items_df.set_index(index_df.index.values)

,Purchase Count,Item Price,Purchase Value
"(92, Final Critic)",13,$4.61,$59.99
"(178, Oathbreaker, Last Hope of the Breaking Storm)",12,$4.23,$50.76
"(145, Fiery Glass Crusader)",9,$4.58,$41.22
"(132, Persuasion)",9,$3.22,$28.99
"(108, Extraction, Quickblade Of Trembling Hands)",9,$3.53,$31.77


## Most Profitable Items

* Reused base list from above in order to create new df based on a different sort

* Sorted based on (Total Purchase Value)

* Used same formatting as above

In [438]:
profit_index_df = pd.DataFrame(item_grouping['Price'].sum()).sort_values(by=['Price'],ascending=False).head()
profit_items_df = pd.DataFrame(product_list,columns=['Purchase Count','Item Price','Purchase Value']).sort_values(by=['Purchase Value'],ascending=False).head()
for x in range(1,3):
    profit_df_col = profit_items_df.columns[x]
    profit_items_df[profit_df_col] = profit_items_df[profit_df_col].map("${:,.2f}".format)
profit_items_df.set_index(profit_index_df.index.values)

,Purchase Count,Item Price,Purchase Value
"(92, Final Critic)",13,$4.61,$59.99
"(178, Oathbreaker, Last Hope of the Breaking Storm)",12,$4.23,$50.76
"(82, Nirvana)",9,$4.90,$44.10
"(145, Fiery Glass Crusader)",9,$4.58,$41.22
"(103, Singed Scalpel)",8,$4.35,$34.80


In [439]:
%reset